In [1]:
import sys
sys.path.append("..") 
from utils.surface_gases import *

In [2]:
COUPLER_options = {}

COUPLER_options['Phi_global'] = 1.0e-19
COUPLER_options['T_outgas'] = 3000.0

COUPLER_options['radius'] = 6.371e6
COUPLER_options['mass']   = 5.972e24
COUPLER_options['gravity'] = 6.67e-11 * COUPLER_options['mass'] / (COUPLER_options['radius']**2)
COUPLER_options['hydrogen_earth_oceans'] = 1.0
COUPLER_options['CH_ratio'] = 2.0
COUPLER_options['fO2_shift_IW'] = 5.0
COUPLER_options['nitrogen_ppmw'] = 2.0
COUPLER_options['planet_coresize'] = 0.55

solvevol_vols = ['H2O', 'CO2', 'N2', 'H2', 'CO', 'CH4']
for v in solvevol_vols:
    COUPLER_options[v+"_included"] = 0
    COUPLER_options[v+"_pa"] = 0.0

COUPLER_options["H2O_included"] = 1
COUPLER_options["CO2_included"] = 1
COUPLER_options["N2_included"]  = 1

COUPLER_options["H2O_initial_bar"] = 1.0
COUPLER_options["CO2_initial_bar"] = 0.0
COUPLER_options["N2_initial_bar"]  = 0.0



In [3]:
COUPLER_options["mantle_mass"] = calc_mantle_mass(COUPLER_options)

In [4]:
solvevol_target = solvevol_get_target_from_params(COUPLER_options)
# solvevol_target = solvevol_get_target_from_pressures(COUPLER_options)

print("Targets:")
print(solvevol_target)

Targets:
{'H': 1.55e+20, 'C': 3.1e+20, 'N': 8.061672162064889e+18}


In [5]:
p_d = solvevol_equilibrium_atmosphere(solvevol_target, COUPLER_options)

print(p_d)

p_bar = {}
for v in volatile_species:
    p_bar[v] = p_d[v+"_atm_bar"]

mubar = 0.0
ptot = 0.0
print("Partial pressures")
for v in solvevol_vols:
    print("    %s : %.1f bar"%(v, p_bar[v]))
    ptot += p_bar[v]
    mubar += p_bar[v]*molar_mass[v]
mubar /= ptot

print("mubar =",mubar)

m_d = {}
for v in solvevol_vols:
    m_d[v] = p_bar[v]*1.0e5 * 4 * 3.14 * (molar_mass[v]/mubar) * COUPLER_options["radius"]**2.0 / COUPLER_options["gravity"]

# print(solvevol_atmosphere_mass(p_bar, COUPLER_options))

print(" ")
print(m_d)

H_kg = (2*m_d["H2O"]/molar_mass["H2O"] + 2*m_d["H2"]/molar_mass["H2"] + 4*m_d["CH4"]/molar_mass["CH4"]) *molar_mass['H']
C_kg = (m_d["CO2"]/molar_mass["CO2"] + m_d["CO"]/molar_mass["CO"] + m_d["CH4"]/molar_mass["CH4"]) * molar_mass['C']
N_kg = m_d["N2"]

print(H_kg, C_kg, N_kg)


{'M_atm': 0.0, 'P_surf': 486.5893074009107, 'H2O_atm_bar': 363.2794718184761, 'H2O_atm_kg': 1.3851033730224664e+21, 'H2O_liquid_kg': 4025.750477365765, 'H2O_solid_kg': 0.0, 'H2O_total_kg': 1.3851033730224664e+21, 'CO2_atm_bar': 121.95011492431834, 'CO2_atm_kg': 1.1358837732008081e+21, 'CO2_liquid_kg': 22.208193767394164, 'CO2_solid_kg': 0.0, 'CO2_total_kg': 1.1358837732008081e+21, 'H2_atm_bar': 0.0, 'H2_atm_kg': 0.0, 'H2_liquid_kg': 0.0, 'H2_solid_kg': 0.0, 'H2_total_kg': 0.0, 'CH4_atm_bar': 0.0, 'CH4_atm_kg': 0.0, 'CH4_liquid_kg': 0.0, 'CH4_solid_kg': 0.0, 'CH4_total_kg': 0.0, 'CO_atm_bar': 0.0, 'CO_atm_kg': 0.0, 'CO_liquid_kg': 0.0, 'CO_solid_kg': 0.0, 'CO_total_kg': 0.0, 'N2_atm_bar': 1.3597206581162269, 'N2_atm_kg': 8.061672162041967e+18, 'N2_liquid_kg': 0.02229789108420262, 'N2_solid_kg': 0.0, 'N2_total_kg': 8.061672162041967e+18, 'H2O_mr': 0.7465833430638106, 'CO2_mr': 0.25062226618934147, 'H2_mr': 0.0, 'CH4_mr': 0.0, 'CO_mr': 0.0, 'N2_mr': 0.002794390746847887, 'H_atm_kg': 1.550

In [6]:
H_mol = H_kg / (1.0e-3)
H_oc  = H_mol / (7.69e22 * 2.0)


print(H_oc)
print("C/H = %.4f" % (C_kg/H_kg))

1.0072914278698855
C/H = 2.0000
